In [46]:
import pystac_client
import os
import boto3
from obstore.store import S3Store
from obstore.auth.boto3 import Boto3CredentialProvider
from virtualizarr.registry import ObjectStoreRegistry
from obstore.store import S3Store
from virtualizarr import open_virtual_dataset
from virtual_tiff import VirtualTIFF
import xarray as xr
import pyproj
import xproj
import rasterix
import logging
import morecantile
import shapely
from shapely.ops import transform
from pyproj import Transformer
from shapely import wkt

In [17]:
# Connect to Earth Search STAC API
catalog = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")

california_bbox = [-124.4096, 32.5343, -114.1312, 42.0095]

search = catalog.search(
    collections=["landsat-c2-l2"],  # Landsat Collection 2 Level-2
    bbox=california_bbox,
    datetime="2025-10-01/2025-10-10",
    max_items=10
)

items = list(search.items())
item = items[0]
item

<Item id=LC09_L2SP_045034_20251009_02_T2>

In [18]:
scheme = "s3://"
bucket = "usgs-landsat"
region = "us-west-2"

In [19]:
profile = "impactnew"
os.environ["AWS_PROFILE"] = profile

In [20]:
session = boto3.Session()
object_store = S3Store(
    bucket=bucket,
    region=region,
    request_payer=True,
    credential_provider=Boto3CredentialProvider(session=session),
)
registry = ObjectStoreRegistry({f"{scheme}{bucket}": object_store})

In [50]:
asset = item.assets["red"]
parser=VirtualTIFF(ifd=0)
manifest_store = parser(url=asset.href, registry=registry)
#ds = open_virtual_dataset(
#    url=asset.href,
#    registry=registry,
#    parser=VirtualTIFF(ifd=0)
#)
ds = xr.open_dataset(manifest_store, engine="zarr", consolidated=False, zarr_format=3)
ds

/Users/seanharkins/projects/zarr-datafusion-examples/.venv/lib/python3.13/site-packages/zarr/codecs/numcodecs/_codecs.py:139: ZarrUserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


<xarray.Dataset> Size: 122MB
Dimensions:  (y: 7871, x: 7751)
Dimensions without coordinates: y, x
Data variables:
    0        (y, x) uint16 122MB ...

In [51]:
da = ds["0"].proj.assign_crs(spatial_ref=item.properties["proj:code"])
da

<xarray.DataArray '0' (y: 7871, x: 7751)> Size: 122MB
[61008121 values with dtype=uint16]
Coordinates:
  * spatial_ref  int64 8B 0
Dimensions without coordinates: y, x
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:32610)
Attributes:
    citation:                    WGS 84 / UTM zone 10N
    geog_angular_units:          9102
    geog_citation:               WGS 84
    model_type:                  1
    proj_linear_units:           9001
    projected_type:              32610
    raster_type:                 2
    model_pixel_scale:           [30.0, 30.0, 0.0]
    model_tiepoint:              [0.0, 0.0, 0.0, 323400.0, 4265400.0, 0.0]
    photometric_interpretation:  1
    gdal_no_data:                0

In [52]:
da = da.pipe(rasterix.assign_index)
da


<xarray.DataArray '0' (y: 7871, x: 7751)> Size: 122MB
[61008121 values with dtype=uint16]
Coordinates:
  * y            (y) float64 63kB 7.87e+03 7.871e+03 ... 1.574e+04 1.574e+04
  * x            (x) float64 62kB 0.0 1.0 2.0 ... 7.748e+03 7.749e+03 7.75e+03
  * spatial_ref  int64 8B 0
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:32610)
  ┌ x            RasterIndex (crs=EPSG:32610)
  └ y
Attributes:
    citation:                    WGS 84 / UTM zone 10N
    geog_angular_units:          9102
    geog_citation:               WGS 84
    model_type:                  1
    proj_linear_units:           9001
    projected_type:              32610
    raster_type:                 2
    model_pixel_scale:           [30.0, 30.0, 0.0]
    model_tiepoint:              [0.0, 0.0, 0.0, 323400.0, 4265400.0, 0.0]
    photometric_interpretation:  1
    gdal_no_data:                0

In [54]:
granule_bbox = morecantile.commons.BoundingBox(*item.bbox)
tms = morecantile.tms.get("WebMercatorQuad")
tiles = list(tms.tiles(*granule_bbox, zooms=10))

In [55]:
tile_bbox = tms.xy_bounds(tiles[0])
tile_bbox_geom = shapely.geometry.box(*tile_bbox)
utm_transformer = Transformer.from_crs("EPSG:3857", item.properties["proj:code"], always_xy=True).transform
tile_bbox_utm = transform(utm_transformer, tile_bbox_geom)
tile_bbox_utm.bounds

(311379.69884914777, 4237611.325443612, 342734.06888959487, 4268841.942051003)

In [56]:
left, bottom, right, top = tile_bbox_utm.bounds
da_tile = da.sel(x=slice(left, right), y=slice(top, bottom))
da_tile

<xarray.DataArray '0' (y: 0, x: 0)> Size: 0B
[0 values with dtype=uint16]
Coordinates:
  * y            (y) float64 0B 
  * x            (x) float64 0B 
  * spatial_ref  int64 8B 0
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:32610)
  ┌ x            RasterIndex (crs=EPSG:32610)
  └ y
Attributes:
    citation:                    WGS 84 / UTM zone 10N
    geog_angular_units:          9102
    geog_citation:               WGS 84
    model_type:                  1
    proj_linear_units:           9001
    projected_type:              32610
    raster_type:                 2
    model_pixel_scale:           [30.0, 30.0, 0.0]
    model_tiepoint:              [0.0, 0.0, 0.0, 323400.0, 4265400.0, 0.0]
    photometric_interpretation:  1
    gdal_no_data:                0